In [7]:
import pandas as pd
import neurokit2 as nk
import numpy as np
import yaml

config = yaml.safe_load(open("config.yaml"))

# get_file_rows_in_interval()

In [6]:
import pandas as pd
from datetime import datetime, timedelta
from ecg_preproc import get_file_rows_in_interval
import yaml

config = yaml.safe_load(open("config.yaml"))

# Define test timestamps
file_start = datetime(2024, 9, 12, 8, 0, 0)
file_end = datetime(2024, 9, 12, 9, 0, 0)

# Define intervals to test
intervals = [
    (datetime(2024, 9, 12, 7, 50, 0), datetime(2024, 9, 12, 8, 10, 0)),  # Interval starts before, ends inside
    (datetime(2024, 9, 12, 8, 20, 0), datetime(2024, 9, 12, 8, 40, 0)),  # Interval fully inside
    (datetime(2024, 9, 12, 8, 50, 0), datetime(2024, 9, 12, 9, 10, 0)),  # Interval ends after
    (datetime(2024, 9, 12, 7, 0, 0), datetime(2024, 9, 12, 10, 0, 0)),    # Interval contains full file
    (datetime(2024, 9, 12, 8, 0, 0), datetime(2024, 9, 12, 9, 0, 0)),    # Exact match
    (datetime(2024, 9, 12, 9, 10, 0), datetime(2024, 9, 12, 9, 20, 0))   # Interval completely outside
]

# Test the function on different intervals
results = []
for interval_start, interval_end in intervals:
    result = get_file_rows_in_interval(file_start, file_end, interval_start, interval_end)
    if not result:
        result = {}
        result['start_time'] = None
        result['end_time'] = None
        result['start_idx'] = None
        result['end_idx'] = None
    results.append({
        'interval_start': interval_start,
        'interval_end': interval_end,
        'start_time': result['start_time'],
        'end_time': result['end_time'],
        'start_idx': result['start_idx'],
        'end_idx': result['end_idx']
    })
file_rows = int((file_end - file_start).seconds * config['ecg_hz'])  # could be off slightly (less than one second)
       
print(f'File length: {file_rows}')
# Convert to DataFrame for display
df_results = pd.DataFrame(results)
df_results['idx_duration'] = (df_results['end_idx'] - df_results['start_idx']) / config['ecg_hz'] /60
df_results

File length: 460800


,interval_start,interval_end,start_time,end_time,start_idx,end_idx,idx_duration
0,2024-09-12 07:50:00,2024-09-12 08:10:00,2024-09-12 08:00:00,2024-09-12 08:10:00,0.0,76800.0,10.0
1,2024-09-12 08:20:00,2024-09-12 08:40:00,2024-09-12 08:20:00,2024-09-12 08:40:00,153600.0,307200.0,20.0
2,2024-09-12 08:50:00,2024-09-12 09:10:00,2024-09-12 08:50:00,2024-09-12 09:00:00,384000.0,460800.0,10.0
3,2024-09-12 07:00:00,2024-09-12 10:00:00,2024-09-12 08:00:00,2024-09-12 09:00:00,0.0,460800.0,60.0
4,2024-09-12 08:00:00,2024-09-12 09:00:00,2024-09-12 08:00:00,2024-09-12 09:00:00,0.0,460800.0,60.0
5,2024-09-12 09:10:00,2024-09-12 09:20:00,NaT,NaT,NaN,NaN,NaN


In [9]:
import pandas as pd
import os
from datetime import datetime, timedelta
from ecg_preproc import get_file_rows_in_interval, get_filename_end_ts, get_filename_start_ts, valid_filename

pid = "9552"
ecg_dir = os.path.join("data/proc_ecg", pid)

interval_start = datetime(2022, 2, 19, 10, 30, 0)
interval_end = datetime(2022, 2, 19, 11, 15, 0)

file_segments = []
for file in os.listdir(ecg_dir):
    if not valid_filename(file, 'signal'):
        print(f"Skipping {file}")
        continue
    file_start = get_filename_start_ts(file)
    file_end = get_filename_end_ts(file)
    file_rows = get_file_rows_in_interval(file_start=file_start, 
                                          file_end=file_end,
                                          interval_start=interval_start,
                                          interval_end=interval_end)
    if file_rows:
        file_segments.append({
            'file': file,
            'start_time': file_rows['start_time'],
            'end_time': file_rows['end_time'],
            'start_idx': file_rows['start_idx'],
            'end_idx': file_rows['end_idx']
        })
        
df_file_segments = pd.DataFrame(file_segments)
df_file_segments = df_file_segments.sort_values(by='start_time')
display(df_file_segments)
    


Skipping ECG_10009_phase_20220114_170000_20220114_180000.parquet
Skipping ECG_9552_rpeaks_20220226_020000_20220226_030000.parquet
Skipping ECG_10009_meta_20220123_220000_20220123_230000_100_0_0.parquet
Skipping ECG_9552_rpeaks_20220304_140000_20220304_150000.parquet
Skipping ECG_9552_meta_20220225_160000_20220225_170000_28_7100_0.parquet
Skipping ECG_9552_qrs_20220301_210000_20220301_220000.parquet
Skipping ECG_9552_qrs_20220304_120000_20220304_130000.parquet
Skipping ECG_9552_qrs_20220226_040000_20220226_050000.parquet
Skipping ECG_9552_phase_20220223_180000_20220223_190000.parquet
Skipping ECG_9552_rpeaks_20220222_070000_20220222_080000.parquet
Skipping ECG_9552_meta_20220303_200000_20220303_210000_98_200_0.parquet
Skipping ECG_9552_meta_20220219_190000_20220219_200000_96_400_0.parquet
Skipping ECG_10009_meta_20220123_080000_20220123_090000_62_3800_0.parquet
Skipping ECG_9552_phase_20220227_150000_20220227_160000.parquet
Skipping ECG_9552_meta_20220225_220000_20220225_230000_98_200_0

,file,start_time,end_time,start_idx,end_idx
1,ECG_9552_signal_20220219_100000_20220219_11000...,2022-02-19 10:30:00,2022-02-19 11:00:00,230400,460800
0,ECG_9552_signal_20220219_110000_20220219_12000...,2022-02-19 11:00:00,2022-02-19 11:15:00,0,115200
